In [17]:
import warnings
# Data related
import numpy as np 
import pandas as pd
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline

# sklearn models
import xgboost as xgb

In [18]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
# Import custom functions
from utils_functions import *

## 1. Load data 

In [19]:
df = pd.read_csv('../data/processed/train_variants_text.csv')
df['Text'] = df['Text'].astype(str)

## 2. Split into Train amd Validation data

In [20]:
X_tr, X_val, y_tr, y_val = split_data(df,
                                      'Text',
                                      'Class',
                                      0.1,
                                      0,
                                      stratify='Class')

## 3. Training

## 3.1 Bag-of-words+xgboost

In [22]:
clf = Pipeline([('vect', CountVectorizer(preprocessor=clean_text_stemmed, stop_words =stop_words)),
                ('tfidf', TfidfTransformer()),
                ('clf',  xgb.XGBClassifier(objective="multi:softprob", random_state=42))])                           

In [25]:
clf.fit(X_tr, y_tr)  
predicted = clf.predict(X_val)
acc=np.mean(predicted == y_val)
print(acc)

0.6756756756756757


## 3.2 Word2vec+xgboost

In [ ]:
# Use document df
w2vec = get_word2vec(
    MySentences(
        df['Text'].values, 
    ),
    'w2vmodel'
)

w2vmodel not found. training model


In [ ]:
clf = Pipeline([('vect', MeanEmbeddingVectorizer(w2vec)),
                ('clf',  xgb.XGBClassifier(objective="multi:softprob", random_state=42))])                           

In [16]:
clf.fit(X_tr, y_tr)  
predicted = clf.predict(X_val)
acc=np.mean(predicted == y_val)
print(acc)

0.6186186186186187
